<a href="https://colab.research.google.com/github/RavenPillmann/ML/blob/master/Introduction_to_DCGANS_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
from io import BytesIO
import cv2
# from PIL import Image
import PIL
from IPython.display import clear_output, Image, display


In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255., x_test / 255.

In [3]:
y_train.shape

(60000,)

In [4]:
# convert y_train and y_test to one-hot-encoded
one_hot_y_train = np.zeros((11, y_train.shape[0]))
one_hot_y_test = np.zeros((11, y_test.shape[0]))

for i in range(y_train.shape[0]):
  val = y_train[i]
  one_hot_y_train[val][i] = 1
  
for i in range(y_test.shape[0]):
  val = y_test[i]
  one_hot_y_test[val][i] = 1
  
  
print(y_train)

[5 0 4 ... 5 6 8]


In [5]:
print(one_hot_y_train)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
disc_dropout = 0.5

def discriminator():
  # Using strides rather than max-pooling. Supposedly, this works better
  model = tf.keras.models.Sequential()
  
  model.add(tf.keras.layers.Conv2D(8, kernel_size=3, input_shape=(28, 28, 1), padding='same'))
#   model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(rate=disc_dropout))
  
  model.add(tf.keras.layers.Conv2D(16, kernel_size=3, strides=2, padding='same'))
#   model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(rate=disc_dropout))
  
  model.add(tf.keras.layers.Conv2D(32, kernel_size=3, strides=2, padding='same'))
#   model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(rate=disc_dropout))
  
  model.add(tf.keras.layers.Conv2D(64, kernel_size=3, strides=2, padding='same'))
#   model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(rate=disc_dropout))
  
  model.add(tf.keras.layers.Flatten())
  
  model.add(tf.keras.layers.Dense(100))
  model.add(tf.keras.layers.Activation('relu'))
  model.add(tf.keras.layers.Dropout(rate=disc_dropout))
  
  model.add(tf.keras.layers.Dense(11))
  model.add(tf.keras.layers.Activation('softmax'))
  
  return model
  

In [7]:
disc_model = discriminator()
disc_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 8)         80        
_________________________________________________________________
activation (Activation)      (None, 28, 28, 8)         0         
_________________________________________________________________
dropout (Dropout)            (None, 28, 28, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 16)        1168      
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)  

In [0]:
gen_dropout = 0.5

def generator():
  # Noise vector to 28 x 28 x 1
    
  model = tf.keras.models.Sequential()
  
  model.add(tf.keras.layers.Dense(7*7*256, input_dim=100))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  
  model.add(tf.keras.layers.Reshape((7, 7, 256)))
  model.add(tf.keras.layers.Dropout(gen_dropout))

  model.add(tf.keras.layers.UpSampling2D())
  model.add(tf.keras.layers.Conv2DTranspose(128, 5, padding='same'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  
  model.add(tf.keras.layers.UpSampling2D())
  model.add(tf.keras.layers.Conv2DTranspose(64, 5, padding='same'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  
  model.add(tf.keras.layers.Conv2DTranspose(32, 5, padding='same'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Activation('relu'))
  
  model.add(tf.keras.layers.Conv2DTranspose(1, 5, padding='same'))
  model.add(tf.keras.layers.Activation('sigmoid'))
  
  
#   model.add(tf.keras.layers.Dense(100, input_shape=(100, 1)))
#   model.add(tf.keras.layers.BatchNormalization())
#   model.add(tf.keras.layers.Activation('relu'))
  
#   model.add(tf.keras.layers.Dense(1000))
#   model.add(tf.keras.layers.BatchNormalization())
#   model.add(tf.keras.layers.Activation('relu'))
  
#   model.add(tf.keras.layers.Dense(28 * 28 * 1))
#   model.add(tf.keras.layers.BatchNormalization())
#   model.add(tf.keras.layers.Activation('relu'))
  
#   model.add(tf.keras.layers.Reshape((28, 28, 1), input_shape=(28 * 28 * 1, 1)))
#   model.add(tf.keras.layers.Activation('sigmoid')) # Makes it so that output is between 0 and 1
  
  return model

In [9]:
# noise = np.random.uniform(-1, 1, 100)

gen_model = generator()
gen_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_v1 (Batc (None, 12544)             50176     
_________________________________________________________________
activation_6 (Activation)    (None, 12544)             0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 256)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 7, 7, 256)         0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 128)       819328    
__________

In [0]:
discriminator_optimizer = tf.keras.optimizers.RMSprop(lr=4e-4)
# discriminator_optimizer = tf.keras.optimizers.Adagrad(lr=8e-5)

discriminator_model = tf.keras.models.Sequential()
discriminator_model.add(disc_model)
discriminator_model.compile(loss='categorical_crossentropy', optimizer=discriminator_optimizer, metrics=['accuracy'])

In [0]:
adversarial_optimizer = tf.keras.optimizers.RMSprop(lr=1e-4)
# adversarial_optimizer = tf.keras.optimizers.Adagrad(lr=4e-5)

adversarial_model = tf.keras.models.Sequential()
adversarial_model.add(gen_model)
adversarial_model.add(disc_model)
adversarial_model.compile(loss='categorical_crossentropy', optimizer=adversarial_optimizer, metrics=['accuracy'])

In [0]:
# /device:GPU:0
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# tf.keras.backend.set_session(sess)

In [13]:
# Training the discriminator and generator

# TODO: Need to modify noise so that it includes the conditions

number_epochs = 20000
batch_size = 64

disc_loss = np.array([0., 0.])
adv_loss = np.array([0., 0.])

for i in range(number_epochs):
  indices = np.random.randint(0, x_train.shape[0], size=batch_size)
  real_images = x_train[indices, :, :]
  real_y = one_hot_y_train[:, indices]

  noise = np.random.uniform(-1., 1., size=[batch_size, 89])
  noise = np.concatenate((noise, real_y.T), axis=1)
  
  fake_images = gen_model.predict(noise)
  fake_images = fake_images[:, :, :, 0]
    
  x = np.concatenate((real_images, fake_images)).reshape(-1, 28, 28, 1)
#   y = one_hot_y_train[:, indices]
  
  fake_y = np.vstack((np.zeros((10, batch_size)), np.ones((1, batch_size))))

#   y[batch_size:, :] = 0
  y = np.hstack((real_y, fake_y))

  discriminator_model.trainable=True
  disc_loss += np.array(discriminator_model.train_on_batch(x, y.T))

#   adv_y = np.ones([batch_size, 1]) # Needs to be changed... I guess maybe random numbers???
  random_numbers = np.random.randint(0, 10, size=batch_size)
  
  adv_y = np.zeros((11, batch_size))
  
  for j in range(batch_size):
    adv_y[random_numbers[j], j] = 1
    
  adv_y = adv_y.T
  
  # TODO: Need to update this now as well

  noise = np.random.uniform(-1., 1., size=[batch_size, 89])
  noise = np.concatenate((noise, adv_y), axis=1)
  
  discriminator_model.trainable=False
  adv_loss += np.array(adversarial_model.train_on_batch(noise, adv_y))
  
  if i % 100 == 0:
    print("disc_loss:", (disc_loss / (i+1)), "adv_loss:", (adv_loss / (i+1)))

Instructions for updating:
Use tf.cast instead.
disc_loss: [2.38102484 0.109375  ] adv_loss: [2.41674471 0.078125  ]
disc_loss: [1.36326709 0.54873144] adv_loss: [2.613415   0.04857673]
disc_loss: [1.18575039 0.60747046] adv_loss: [2.19370873 0.20988806]
disc_loss: [1.04521409 0.65391404] adv_loss: [1.72777761 0.37941238]
disc_loss: [0.95362687 0.68524002] adv_loss: [1.3912066  0.50448099]
disc_loss: [0.98999872 0.67282622] adv_loss: [1.21361512 0.57650324]
disc_loss: [1.07344099 0.63060264] adv_loss: [1.21214841 0.55355657]
disc_loss: [1.10960192 0.60497281] adv_loss: [1.23526521 0.51228156]
disc_loss: [1.12649434 0.58806375] adv_loss: [1.24695013 0.48304853]
disc_loss: [1.13146934 0.57815968] adv_loss: [1.25065532 0.46318327]
disc_loss: [1.12894485 0.57286464] adv_loss: [1.25918867 0.44204233]
disc_loss: [1.12432475 0.56957453] adv_loss: [1.26215458 0.425948  ]
disc_loss: [1.1196531  0.56639649] adv_loss: [1.26362975 0.41297617]
disc_loss: [1.11512484 0.56265613] adv_loss: [1.2615473

In [0]:
def showarray(a, fmt='jpeg'):
  a = np.uint8(np.clip(a, 0, 255))
  f = BytesIO()
  PIL.Image.fromarray(a).save(f, fmt)
  display(Image(data=f.getvalue()))

In [43]:
random_input = np.random.uniform(-1., 1., size=[1, 89])
number = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
print(number.shape, random_input.shape)
random_input = np.hstack((random_input, number))
generated_image = gen_model.predict(random_input)[0, :, :, 0]
generated_image = generated_image * 255.

# generated_image = cv2.resize(generated_image, (0, 0), fx=2., fy=2.) # Back to full scale

%matplotlib inline
# im = Image.fromarray(generated_image)
# im.show()
showarray(generated_image)

(1, 11) (1, 89)


In [0]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())